## Load the model

In [1]:
from keras.layers import Dense, Input, GRU, Embedding
from keras.models import Model

MAX_SEQUENCE_LENGTH = 200
EMBEDDING_DIM = 300
word_index_length = 374000

embedding_layer = Embedding(word_index_length + 1,
                            EMBEDDING_DIM,
                            #weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = GRU(128, dropout_W=0.2, dropout_U=0.2)(embedded_sequences)
preds = Dense(2, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

Using TensorFlow backend.


## Load Weights

In [2]:
model.load_weights('/home/dan-laptop/Insight/stackex_sum/Data_and_Models/stackex_gru.h5')

## Test model

In [3]:
import numpy as np
test_me = np.zeros((200,1))

In [4]:
test_me[-5:] = np.array([[23,48,65,78,120]]).T
test_me.T

array([[   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
      

In [7]:
model.predict(test_me.T)[:,1]

0.02240447

In [14]:
pwd

'/home/dan-laptop/Insight/stackex_sum/Modeling'

## Load model preprocessing and test

In [28]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
import six.moves.cPickle as cPickle
import os

save_dir = '../Data_and_Models'
tokenizer_fname = "rnn_tokenizer.pkl"

tokenizer = cPickle.load(open(os.path.join(save_dir, tokenizer_fname), 'rb'), encoding='latin1')

In [84]:
#just_tok = [['user1571934', 'no', "ed's", 'code', 'is', 'correct', 'works', 'on', '0', '14', '0'], ['what', 'version', 'of', 'pandas', 'are', 'you', 'using'], ['the', 'other', 'option', 'is', 'not', 'to', 'use', 'usecols', 'and', 'just', 'select', 'the', 'columns', 'you', 'want', 'after', 'reading']]
just_tok = [['the', 'other', 'option', 'is', 'not', 'to', 'use', 'usecols', 'and', 'just', 'select', 'the', 'columns', 'you', 'want', 'after', 'reading']]


just_tok = [tokenizer.texts_to_sequences(x) for x in just_tok]
if len(just_tok[0]) > 1:
    temp = []
    for items in just_tok: temp.append([x[0] for x in items if len(x)>0])
    just_tok = temp
else:
    just_tok = [[x[0] for x in just_tok if len(x)>0]]

padded_seq = pad_sequences(just_tok,maxlen=MAX_SEQUENCE_LENGTH)
seq = np.array(padded_seq)
print(seq.shape)
y_pred = model.predict(seq)
y_pred = y_pred[:,1] - y_pred[:,0]

(1, 200)


In [85]:
y_pred

array([-0.72633827], dtype=float32)